In [12]:
import pandas as pd
import numpy as np
from skimage import transform, io, exposure
from PIL import Image
from os.path import join
import os
from random import shuffle
import warnings
warnings.filterwarnings("ignore")

PATH = os.path.join("dataset", "images")
PATH_AUGMENTED = os.path.join("dataset", "data_augmented")

In [8]:
df = pd.read_csv("dataset/name_images.csv", index_col=0)
df.head()

,filename,label
0,DIESTRO_142_4.jpg,DIESTRO
1,DIESTRO_143_4.jpg,DIESTRO
2,DIESTRO_144_4.jpg,DIESTRO
3,DIESTRO_145_4.jpg,DIESTRO
4,DIESTRO_150_4.jpg,DIESTRO


In [13]:
new_name_images = []
for index, features in df.iterrows():
    filename = features["filename"]
    name = filename[:-4]
    label = features["label"]
    image = (exposure.equalize_hist(io.imread(os.path.join(PATH, filename), as_gray=True)[:800,:800]) *255).astype(np.uint8)
    for rotation in [0, 90, 180, 270]: 
        rotated_image = transform.rotate(image, rotation, preserve_range=True).astype(pd.np.uint8)
        quadrants = [rotated_image[:400, 400:], 
                     rotated_image[:400, :400], 
                     rotated_image[400:, :400], 
                     rotated_image[400:, 400:]]
        for idx, quadrant in enumerate(quadrants):
            filename_ =  "{}_{}_{}.jpg".format(name, rotation, idx) 
            io.imsave(os.path.join(PATH_AUGMENTED, filename_), quadrant)
            new_name_images.append({"filename": filename_, "label":label})
            
            flipped_quadrant = quadrant[:,::-1]
            filename_ = "{}_{}_{}_fliped.jpg".format(name, rotation, idx)
            io.imsave(os.path.join(PATH_AUGMENTED, filename_), flipped_quadrant)
            new_name_images.append({"filename": filename_, "label":label})

In [14]:
shuffle(new_name_images)
df = pd.DataFrame(new_name_images)
df.head()

,filename,label
0,PROESTRO_535_1_0_1_fliped.jpg,PROESTRO
1,ESTRO_227_2_90_2_fliped.jpg,ESTRO
2,METAESTRO_153_3_90_0_fliped.jpg,METAESTRO
3,DIESTRO_603_4_90_0_fliped.jpg,DIESTRO
4,ESTRO_547_2_0_1_fliped.jpg,ESTRO


In [15]:
df.to_csv("dataset/name_data_augmented.csv")